In [1]:
import torch
import numpy as np
import torch_geometric
from torch import Tensor
from torch.nn import Linear
from torch_geometric.data import Data, Dataset
from torch_geometric.nn.conv import GATConv
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import DataLoader
from sklearn.metrics import roc_auc_score, RocCurveDisplay
import matplotlib.pyplot as plt
from tqdm import tqdm
#torch.set_default_dtype(torch.float32)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
import gc
torch.cuda.empty_cache()
gc.collect()
#print(torch.cuda.memory_summary())

Using device: cuda


0

In [2]:
data = torch.load("data/act-mooc/graph.pt")
#data = torch.load("data/junyi/graph.pt")

In [3]:
data_hom = data.to_homogeneous()

In [4]:
data = data_hom
# no data features is implemented as just one random number drawn from normal
data.x = torch.randn(data.num_nodes, 1)
data.edge_attr = data.edge_attr.float()
data.time = data.time.float()
data = data.to(device)
data

Data(edge_index=[2, 823498], node_id=[7144], edge_attr=[823498, 4], time=[823498], edge_y=[823498], node_type=[7144], edge_type=[823498], x=[7144, 1])

node_type==1 are resources
let's split randomly by users

In [5]:
time_array = data.time.cpu().numpy()
quantile_70 = np.quantile(time_array, 0.7)
quantile_85 = np.quantile(time_array, 0.85)

# Create masks for splitting the data
train_mask = time_array < quantile_70
val_mask = time_array < quantile_85
test_mask = np.ones_like(time_array, dtype=bool)

# Function to create a new Data object from the original data and a mask
def create_subset(data, mask):
    # Filter data using the mask
    subset = Data()
    for key, item in data:
        if key in ['node_id', 'node_type', 'x']:
            subset[key] = item
        elif key in ['edge_index']:
            subset[key] = item[:,mask]
        else:
            subset[key] = item[mask]
    return subset

# Create subsets
train_data = create_subset(data, train_mask)
val_data = create_subset(data, val_mask)
test_data = create_subset(data, test_mask)

In [6]:
def create_mask_in_batches(edge_index, nodes, batch_size):
    mask = torch.zeros(edge_index.size(1), dtype=torch.bool, device=device)
    for i in range(0, len(nodes), batch_size):
        batch_nodes = nodes[i:i + batch_size]
        batch_mask = ((edge_index[0].unsqueeze(1) == batch_nodes).any(dim=1)) | \
                     ((edge_index[1].unsqueeze(1) == batch_nodes).any(dim=1))
        mask |= batch_mask
    return mask

This class iterates over batches of users and returns all edges they are connected with.

In [9]:
class TemporalLoader:
    def __init__(self, data, start_time, batch_size=32):
        self.data = data
        self.time_order = torch.argsort(self.data.time)
        self.start_time = start_time
        self.start_index = torch.nonzero(self.data.time[self.time_order]>=start_time)[0]
        self.index = self.start_index
        self.batch_size = batch_size
        self.length = len(data.edge_y) - self.start_index

    def __iter__(self):
        return self

    def __len__(self):
        return self.length // self.batch_size

    def reset(self):
        self.index = 0

    def __next__(self):
        if self.index + self.batch_size >= self.length:
            self.index = 0
            raise StopIteration

        # these edges are predicted
        mask = torch.zeros(self.data.edge_index.size(1), dtype=torch.bool).to(device)
        mask[self.time_order[self.index+self.start_index:self.index+self.start_index+self.batch_size]] = 1

        # but add neighbors that may be useful for the prediction = edges containing the same IDs in the time before
        before_mask = torch.zeros(self.data.edge_index.size(1), dtype=torch.bool).to(device)
        before_mask[self.time_order[:self.index+self.start_index]] = 1
        first_time = self.data.time[self.time_order[self.index+self.start_index]]

        edge_nodes = self.data.edge_index[:,mask].unique()
        neighbor_mask = create_mask_in_batches(self.data.edge_index, edge_nodes, 256)

        mask |= (neighbor_mask & before_mask)
        
        batch = create_subset(self.data, mask)
        batch.edge_y[batch.time < first_time] = -1
        self.index += self.batch_size
        return batch

In [11]:
batch_size=256
train_loader = TemporalLoader(train_data, 0, batch_size=batch_size)
val_loader = TemporalLoader(val_data, quantile_70, batch_size=batch_size)
test_loader = TemporalLoader(test_data, quantile_85, batch_size=batch_size)

In [12]:
for batch in val_loader:
    assert batch.time[batch.edge_y>=0].min() >= quantile_70
    assert batch.time[batch.edge_y>=0].max() < quantile_85

In [13]:
for batch in test_loader:
    assert batch.time[batch.edge_y>=0].min() >= quantile_85

In [14]:
class TimeEncoder(torch.nn.Module):
    def __init__(self, time_dim):
        super().__init__()
        self.time_dim = time_dim
        self.time_lin = Linear(1, time_dim)

    def forward(self, t: Tensor):
        return self.time_lin(t.view(-1, 1)).cos()

class GATModel(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, edge_dim, time_enc):
        super().__init__()
        self.conv1 = GATConv(in_channels, hidden_channels, edge_dim=edge_dim)
        self.conv2 = GATConv(hidden_channels, out_channels, edge_dim=edge_dim)
        self.time_enc = time_enc

    def forward(self, x: Tensor, edge_index: Tensor, edge_attr: Tensor, t: Tensor) -> Tensor:
        # x: Node feature matrix of shape [num_nodes, in_channels]
        # edge_index: Graph connectivity matrix of shape [2, num_edges]
        # edge_attr: Edge features
        # t: Timestamp of edges
        time_enc = self.time_enc(t)
        edge_attr = torch.cat([time_enc, edge_attr], dim=-1)
        x = self.conv1(x, edge_index, edge_attr).relu()
        x = self.conv2(x, edge_index, edge_attr)
        return x

class LinkPredictor(torch.nn.Module):
    def __init__(self, in_channels, time_enc):
        super().__init__()
        self.time_enc = time_enc
        dim = 2 * in_channels + time_enc.time_dim
        self.lin = Linear(dim, dim)
        self.lin_final = Linear(dim, 1)

    def forward(self, z_src, z_dst, t):
        time_enc = self.time_enc(t)
        input = torch.cat([z_src, z_dst, time_enc], dim=-1)
        h = self.lin(input)
        h = h.relu()
        return self.lin_final(h)

emd_dim = hidden_dim = time_dim = 100

time_enc = TimeEncoder(time_dim).to(device)
gnn = GATModel(data.x.size(1), hidden_dim, emd_dim, time_dim+data.edge_attr.size(1), time_enc).to(device)
link_pred = LinkPredictor(emd_dim, time_enc).to(device)

# Test forward run of the models
batch = next(iter(train_loader)).to(device)
train_loader.reset() # reset
embs = gnn(batch.x, batch.edge_index, batch.edge_attr, batch.time)
print(embs.shape)
link_preds = link_pred(embs[batch.edge_index[0]], embs[batch.edge_index[1]], batch.time)
print(link_preds.shape)

torch.Size([7144, 100])
torch.Size([256, 1])


In [186]:
@torch.no_grad()
def test(gnn, link_pred):
    gnn.eval()
    link_pred.eval()
    scores = []
    for batch in tqdm(test_loader, desc='Validation Batches'):
        if (batch.edge_y>0).any():
            mask = batch.edge_y < 0
            node_embs = gnn(batch.x, batch.edge_index[:,mask], batch.edge_attr[mask], batch.time[mask])
            link_preds = link_pred(node_embs[batch.edge_index[0,~mask]], node_embs[batch.edge_index[1,~mask]], batch.time[~mask]).sigmoid()
            #RocCurveDisplay.from_predictions(batch.edge_y.cpu(), link_preds.cpu())
            #plt.show()
            scores += [roc_auc_score(batch.edge_y[~mask].cpu().numpy(), link_preds.cpu().numpy())]
    return np.mean(scores)

In [187]:
optimizer = torch.optim.Adam(set(time_enc.parameters()) | set(gnn.parameters()) | set(link_pred.parameters()), lr=0.0001)
criterion = torch.nn.BCEWithLogitsLoss()

for epoch in range(0, 20):
    gnn.train()
    link_pred.train()
    
    for batch in tqdm(train_loader, desc='Training Batches'):
        batch.to(device)
        # forward pass with edges that are older than this batch
        mask = batch.edge_y < 0
        node_embs = gnn(batch.x, batch.edge_index[:,mask], batch.edge_attr[mask], batch.time[mask])
        
        # first predict for all dropout edges
        positive_edges = batch.edge_index[:, batch.edge_y == 1]
        pos_out = link_pred(node_embs[positive_edges[0]], node_embs[positive_edges[1]], batch.time[batch.edge_y == 1])
        loss = criterion(pos_out, torch.ones_like(pos_out))

        # sample the same number of 0 edges from edge_index
        negative_indices = torch.nonzero(batch.edge_y == 0).squeeze()
        negative_indices = negative_indices[torch.randperm(negative_indices.size(0))][:positive_edges.size(1)]
        negative_edges = batch.edge_index[:, negative_indices]
        neg_out = link_pred(node_embs[negative_edges[0]], node_embs[negative_edges[1]], batch.time[negative_indices])
        loss += criterion(neg_out, torch.zeros_like(neg_out))

        # backward pass and gradient update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    test_auc = test(gnn, link_pred)
    print(f'Epoch: {epoch}, Train Loss: {loss}, Test AUC: {test_auc}')
    

Validation Batches:   0%|          | 0/120 [00:00<?, ?it/s]
/home/cpsc483_dg963/.conda/envs/mooc/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/cpsc483_dg963/.conda/envs/mooc/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Epoch: 0, Train Loss: 1.4216886758804321, Test AUC: nan


Validation Batches: 100%|██████████| 120/120 [00:02<00:00, 40.07it/s]


Epoch: 1, Train Loss: 1.2756704092025757, Test AUC: 0.6199525336302903


Validation Batches: 100%|██████████| 120/120 [00:02<00:00, 40.15it/s]


...

Epoch: 4, Train Loss: 1.7167401313781738, Test AUC: 0.6553190172262299


Validation Batches: 100%|██████████| 120/120 [00:02<00:00, 41.07it/s]


Epoch: 5, Train Loss: 1.4337031841278076, Test AUC: 0.6524354375715756


Training Batches:  94%|█████████▍| 528/562 [00:11<00:00, 45.49it/s]


KeyboardInterrupt: 

In [ ]:
print(torch.cuda.memory_summary())